# Scale with formula

We want to make F657N image to its equivalent F665N image at count rates. If we can do this, we can perform image_F665N - image_F657N_transformed_to_F665N.

Since,
$$
flam = 10^{-0.4*(mag + 2.406 + 4*log_{10}(ABwave))}
$$

$$
mag = -2.5*log_{10}(countRate) + ZP
$$

By assuming constant flam in two bandpasses,
$$
scale = \frac{countRate_{F1}}{countRate_{F2}} = 10^K
$$

$$
K = 0.4*((ZP_{F1} - ZP_{F2}) + 4*log_{10}(\frac{ABwave_{F1}}{ABwave_{F2}}))
$$

In [ ]:
import numpy as np
import matplotlib.pyplot as plt
from astropy.io import fits
import glob,os,copy
from astropy.wcs import WCS
from photutils.aperture import CircularAperture
from photutils.aperture import aperture_photometry
import pysynphot as S

In [ ]:
filename1 = ['../ASSETS/DRC_F665N/F665N_20200529_IVM_refimage20200529_drc.fits',
             '../ASSETS/REGION/AT2018cow_F665N_20200529_from_offset_xy.dat',
             'F665N',
             'wfc3,uvis2,f665n',
             (6656.,131.,22.721),
            ]
filename2 = ['../ASSETS/DRC_F657N/F657N_20200529_IVM_refimage20200529_drc.fits',
             '../ASSETS/REGION/AT2018cow_F657N_20200529_from_offset_xy.dat',
             'F657N',
             'wfc3,uvis2,f657n',
             (6567.,121.,22.658),
            ]
halpha_rest = 6562.8
zshift = 0.014
halpha_obs = halpha_rest * (1. + zshift)
epoch = '20200529'
regionfile = '../ASSETS/REGION/S01_F665N_20200529_radec.reg'
outnameprefix = '{0}_{1}_sub_{2}'.format(filename1[2],epoch,filename2[2])
outputdestination = '../ASSETS/Halpha/'

In [ ]:
posxy1,posxy2 = [],[]
#####
f = open(filename1[1],'r')
for ii,i in enumerate(f.readlines()):
    if i.split(' ')[0]=='x':
        tx = float(i.split(' ')[1])
    if i.split(' ')[0]=='y':
        ty = float(i.split(' ')[1])
posxy1.append(tx),posxy1.append(ty)
f.close()
#####
f = open(filename2[1],'r')
for ii,i in enumerate(f.readlines()):
    if i.split(' ')[0]=='x':
        tx = float(i.split(' ')[1])
    if i.split(' ')[0]=='y':
        ty = float(i.split(' ')[1])
posxy2.append(tx),posxy2.append(ty)
f.close()
#####
posxy1,posxy2 = np.array(posxy1),np.array(posxy2)
posxy1,posxy2

In [ ]:
##### prepare tmp
t = glob.glob('*')
if 'tmp' in t:
    os.system('rm -R tmp')
os.system('mkdir tmp')

In [ ]:
print(posxy1-posxy2)
print('Implying that two images are aligned.')

In [ ]:
##### F1 = F665N, F2 = F657N
abwave1,_,zp1 = filename1[4]
abwave2,_,zp2 = filename2[4]
K = 0.4 * ((zp1-zp2) + 4.*np.log10(abwave1/abwave2))
scale = np.power(10,K)
scale

In [ ]:
filenamesub = './tmp/{0}_ScaleFormula_drc.fits'.format(outnameprefix)
os.system('cp {0} {1}'.format(filename1[0],filenamesub))
#####
t1 = fits.open(filename1[0])['SCI'].data
t2 = fits.open(filename2[0])['SCI'].data * scale
t3 = fits.open(filenamesub)
tx,ty = posxy1
dx,dy = 100,100
alpha = 0.9
ms = 10
#####
t = t1-t2
t3['SCI'].data = t
t3.writeto(filenamesub,overwrite=True)
#####
plt.figure(figsize=(10,10))
plt.imshow(t,origin='lower',cmap='viridis',vmin=-0.05,vmax=0.05)
plt.plot(tx,ty,'rx',alpha=alpha,ms=ms)
# plt.xlim(tx-dx,tx+dx),plt.ylim(ty-dy,ty+dy)
plt.title('{0} - {1}'.format(filename1[2],filename2[2]))
plt.show()
#####
plt.figure(figsize=(10,10))
plt.imshow(t,origin='lower',cmap='viridis',vmin=-0.05,vmax=0.05)
plt.scatter(tx,ty,s=500,facecolor='None',edgecolor='red',alpha=alpha)
plt.xlim(tx-dx,tx+dx),plt.ylim(ty-dy,ty+dy)
plt.title('{0} - {1}'.format(filename1[2],filename2[2]))
plt.show()
#####
m = np.isfinite(t1)
vmin,vmax = np.percentile(t1[m],5.),np.percentile(t1[m],99.)
plt.figure(figsize=(10,10))
plt.imshow(t1,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
plt.plot(tx,ty,'rx',alpha=alpha,ms=ms)
plt.xlim(tx-dx,tx+dx),plt.ylim(ty-dy,ty+dy)
plt.title(filename1[0])
plt.show()
#####
plt.figure(figsize=(10,10))
plt.imshow(t2,origin='lower',cmap='viridis',vmin=vmin,vmax=vmax)
plt.plot(tx,ty,'rx',alpha=alpha,ms=ms)
plt.xlim(tx-dx,tx+dx),plt.ylim(ty-dy,ty+dy)
plt.title(filename2[0])
plt.show()

# Error and SNR Images

In [ ]:
rn1 = fits.open(filename1[0])[0].header['READNSEC']
rn2 = fits.open(filename2[0])[0].header['READNSEC']
rn1,rn2

In [ ]:
expt1 = fits.open(filename1[0])[0].header['EXPTIME']
expt2 = fits.open(filename2[0])[0].header['EXPTIME']
expt1,expt2

In [ ]:
scale = 1.0828145309286876
var1 = fits.open(filename1[0])['SCI'].data * expt1 + np.power(rn1,2)
var2 = fits.open(filename2[0])['SCI'].data * expt2 + np.power(rn2,2)
var_tot = var1 + var2 * np.power(scale,2)
err_tot = np.sqrt(var_tot)

In [ ]:
filenamesub = './tmp/{0}_ErrTotal_drc.fits'.format(outnameprefix)
os.system('cp {0} {1}'.format(filename1[0],filenamesub))
#####
t3 = fits.open(filenamesub)
t3['SCI'].data = err_tot
t3.writeto(filenamesub,overwrite=True)

In [ ]:
filenamesub = './tmp/{0}_SNRTotal_drc.fits'.format(outnameprefix)
os.system('cp {0} {1}'.format(filename1[0],filenamesub))
#####
t1 = fits.open(filename1[0])['SCI'].data
t2 = fits.open(filename2[0])['SCI'].data * scale
t3 = fits.open(filenamesub)
#####
t = t1-t2
t3['SCI'].data = (t * expt1) / err_tot
t3.writeto(filenamesub,overwrite=True)

In [ ]:
t = glob.glob('./tmp/*')
for i in t:
    os.system('mv {0} {1}'.format(i,outputdestination))

# Source Detection and Segmentation Map

In [ ]:
import os,glob
import numpy as np
from astropy.io import fits
import matplotlib.pyplot as plt
from photutils.segmentation import detect_threshold
from photutils.segmentation import detect_sources
from astropy.stats import gaussian_fwhm_to_sigma
from astropy.convolution import Gaussian2DKernel

In [ ]:
filename = '../ASSETS/Halpha/F665N_20200529_sub_F657N_ScaleFormula_drc.fits'
segmentation_outname = 'halpha_segmentation_drc.fits'
outputfolder = '../ASSETS/Halpha/'
regionfile = '../ASSETS/REGION/AT2018cow_F665N_20200529_from_offset_xy.dat'

In [ ]:
region_xy = [None,None]
t = open(regionfile,'r')
for ii,i in enumerate(t.readlines()):
    if i.split(' ')[0]=='x':
        region_xy[0] = float(i.split(' ')[1])
    if i.split(' ')[0]=='y':
        region_xy[1] = float(i.split(' ')[1])
t.close()
#####
region_xy

In [ ]:
# ##### make tmp folder
# t = glob.glob('*')
# if 'tmp' in t:
#     os.system('rm -R tmp')
# os.system('mkdir tmp')

In [ ]:
fwhm = 1.7
sigma = fwhm * gaussian_fwhm_to_sigma
print(sigma,gaussian_fwhm_to_sigma)

In [ ]:
nsigma = 1.5
npixels = 1
fwhm = 1.7 # pixel
# fwhm = 1.
sigma = fwhm * gaussian_fwhm_to_sigma
size = 3 # size x size kernel
filter_kernel = Gaussian2DKernel(sigma,x_size=size,y_size=size)
connectivity = 8
#####
data = fits.open(filename)['SCI'].data.copy()
error = None
threshold = detect_threshold(data,nsigma=nsigma,error=error)
segm = detect_sources(data,threshold,npixels=npixels,filter_kernel=filter_kernel,connectivity=connectivity)
##### output segmentation image
t = './tmp/{0}'.format(segmentation_outname)
os.system('cp {0} {1}'.format(filename,t))
t2 = fits.open(t)
t2['SCI'].data = segm.copy()
t2.writeto(t,overwrite=True)
#####
# fig,(ax1,ax2) = plt.subplots(1,2,figsize=(20,20))
# ax1.imshow(data,origin='lower',cmap='viridis',vmin=-0.01,vmax=0.01)
# ax1.set_title('Data')
# cmap = segm.make_cmap(seed=123)
# ax2.imshow(segm,origin='lower',cmap=cmap,interpolation='nearest')
# ax2.set_title('Segmentation Image')
#####
dx,dy = 30,30
tx,ty = region_xy
fig,(ax1,ax2) = plt.subplots(1,2,figsize=(20,20))
ax1.imshow(data,origin='lower',cmap='viridis',vmin=-0.01,vmax=0.01)
ax1.scatter(tx,ty,s=100,marker='x',color='r',linewidths=2)
ax1.set_title('Data')
ax1.set_xlim(tx-dx,tx+dy)
ax1.set_ylim(ty-dy,ty+dy)
cmap = segm.make_cmap(seed=123)
ax2.imshow(segm,origin='lower',cmap=cmap,interpolation='nearest')
ax2.scatter(tx,ty,s=100,marker='x',color='r',linewidths=2)
ax2.set_title('Segmentation Image')
ax2.set_xlim(tx-dx,tx+dy)
ax2.set_ylim(ty-dy,ty+dy)

In [ ]:
t2 = t.split('/')[-1]
t3,t4 = '_'.join(t2.split('_')[0:-1]),t2.split('_')[-1]
t3 = t3 + '_nsigma{0}'.format(int(nsigma))
t5 = outputfolder + t3 + '_' + t4
os.system('mv {0} {1}'.format(t,t5))